# Transformer 강의 정리

# Attention Mechanism이란?

- RNN 기반 seq2seq 모델의 문제
    - 하나의 고정된 크기의 벡터에 모든 정보를 압축하려고 하니까 정보 손실이 발생
    - 기울기 소실(Vanishing Gradient) 문제 (RNN의 고질적 문제)
- Attention Mechanism: 디코더에서 출력 단어를 예측하는 매 시점마다 인코더에서의 전체 입력 문장을 다시 한 번 참고한다!
    - $\text{Attention}(Q, K, V)$
        - Q(Query) : 질문 $Q$의 답이 무엇인가? (현재 hidden layer로 날리는 질문)
        - K(Key) : 기존의 데이터의 key → $K$라는 key 에 대하여
        - V(Value) : $K$라는 key에 해당하는 value $V$
        - 즉, Attention의 값은 $K$라는 key에 대하여 $Q$라는 질문을 했을 때 나오는 답(연관성의 정도)을 value $V$와 곱한 값 → Attention Layer로 Encoder에서는 다른 단어와의 관계성 또한 encoding할 수 있음.

# Transformer에서의 Attention

- Multi-head Attention 사용

![http://jalammar.github.io/images/t/Transformer_decoder.png](http://jalammar.github.io/images/t/Transformer_decoder.png)

[http://jalammar.github.io/illustrated-transformer/](http://jalammar.github.io/illustrated-transformer/)

## Self Attention

- Encoder, Decoder Layer 내에 존재하는 attention layer
    - 단, encoder에서는 모든 word에 대해 계산하지만 Decoder는 현재 레이어 이전 값만 계산 → Decoder는 Masked Attention 사용

![https://wikidocs.net/images/page/31379/attention.PNG](https://wikidocs.net/images/page/31379/attention.PNG)

[https://wikidocs.net/31379](https://wikidocs.net/31379)

- 문장을 구성하는 각 단어 간의 상관관계를 구하는 layer
    - Q: 현재 hidden layer에서 관장하는 단어
    - K: 문장 전체 word의 key 값
    - V: key에 해당하는 word value
    - Q, K, V는 모두 같은 출처에서 나온 값
- Self Attention은 현재 문장에서 특정 단어가 문장 전체 word의 key와 어떤 상관관계를 가지고 있는가? 라는 질문에 답할 수 있도록 함

## Encoder-Decoder 간의 Attention

- Encoder의 맥락에서 본 Decoder의 next Hidden Layer 값
    - Q: Decoder의 이전 Layer에서 온 값
    - K: Encoder에서 온 Key 값
    - V: Encoder의 Key에 해당하는 Value

## Multi-Head Attention

![http://jalammar.github.io/images/t/transformer_multi-headed_self-attention-recap.png](http://jalammar.github.io/images/t/transformer_multi-headed_self-attention-recap.png)

[http://jalammar.github.io/illustrated-transformer/](http://jalammar.github.io/illustrated-transformer/)

- **그림 상에서는 Embedding Vector의 크기는 4, Q/K/V Vector의 크기는 3으로 나와있지만 이는 단순화된 이미지 → 실제 Embedding Vector의 크기는 512, Q/K/V Vector의 크기는 64이다.**

$$\text{Attention}(Q, K, V) = \text{softmax}({QK^T \over \sqrt{d_k}})V$$

$$\text{MultiHead}(Q,K,V)=[head_1;\cdots;head_h]W^O \\
head_i=\text{Attention}(RW^Q_i,RW^K_i,RW^V_i)$$

- $R$ : 첫 번째 encoder layer에 들어가는 embedded input / 이전 encoder layer를 지난 결과값
- $\sqrt{d_k}$를 나누는 이유: more stable gradients
- Why Multi-Head?
    - multi-head attention allows the model to jointly attend to information from different representation subspaces at different positions. With a single attention head, averaging inhibits this.
    - Convolution에서 Channel을 여러 개 주는 것과 비슷한 이유라고 이해함

# Transformer Model의 구조

![http://jalammar.github.io/images/t/transformer_resideual_layer_norm_3.png](http://jalammar.github.io/images/t/transformer_resideual_layer_norm_3.png)

[http://jalammar.github.io/illustrated-transformer/](http://jalammar.github.io/illustrated-transformer/)

## Positional Encoding

- 각 단어의 위치 정보를 encoding 하는 Layer

## Encoder Layer

- Self-Attention
- Add & Normalize (ResNet처럼 Residual)
- Feed Foward
    - 입력과 출력의 차원이 동일
- Add & Normalize (Residual)

### Decoder Layer

- Self-Attention
- Add & Normalize
- Encoder-Decoder Attention
- Add & Normalize
- Feed Foward
- Add & Normalize (Residual)

### Linear & Softmax

- 결과적으로 어떤 단어가 나와야 할 지를 결정

# 코드

- [동빈나님 Transformer 코드](https://colab.research.google.com/github/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/Attention_is_All_You_Need_Tutorial_(German_English).ipynb)

## How Translation Works?


In [ ]:
# 번역(translation) 함수
def translate_sentence(sentence, src_field, trg_field, model, device, max_len=50, logging=True):
    model.eval() # 평가 모드

    if isinstance(sentence, str):
        nlp = spacy.load('de')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    # 처음에 <sos> 토큰, 마지막에 <eos> 토큰 붙이기
    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
    if logging:
        print(f"전체 소스 토큰: {tokens}")

    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    if logging:
        print(f"소스 문장 인덱스: {src_indexes}")

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)

    # 소스 문장에 따른 마스크 생성
    src_mask = model.make_src_mask(src_tensor)

    # 인코더(endocer)에 소스 문장을 넣어 출력 값 구하기
    with torch.no_grad():
		# Encoder의 경우 문장 전체 token을 바로 입력으로 넣어준다.
        enc_src = model.encoder(src_tensor, src_mask)

    # 처음에는 <sos> 토큰 하나만 가지고 있도록 하기
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    for i in range(max_len):
        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)

        # 출력 문장에 따른 마스크 생성
        trg_mask = model.make_trg_mask(trg_tensor)

        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)

        # 출력 문장에서 가장 마지막 단어만 사용
        pred_token = output.argmax(2)[:,-1].item()
        trg_indexes.append(pred_token) # 출력 문장에 더하기

        # <eos>를 만나는 순간 끝
        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break

    # 각 출력 단어 인덱스를 실제 단어로 변환
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]

    # 첫 번째 <sos>는 제외하고 출력 문장 반환
    return trg_tokens[1:], attention

''' 
je suis etudient == I am a student

<sos>
-> <sos> I
['I']

-> <sos> I am
['I', 'am']

->
...

-> <sos> I am a student <eos>
['I', 'am' 'a', 'student', '<eos>']
'''



- Transformer 모델은 Encoder-Decoder를 학습시키기 위한 모델
    - 학습이 완료된 후 Translation을 사용할 때는 source string을 Encoder에 넣어 인코딩 시킨 다음 target string의 `<eos>` 에 도달할 때까지 decoder를 돌리는 형태

# 참고 자료

- [https://wikidocs.net/22893](https://wikidocs.net/22893)
- [[동빈나] Attention Is All You Need 논문 리뷰](https://youtu.be/AA621UofTUA)
- [Illustrated Transformer](http://jalammar.github.io/illustrated-transformer/)
    - [한국어판](https://nlpinkorean.github.io/illustrated-transformer/)
- [Attention? Attention!](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html#multi-head-self-attention)